# Import Data

In [1]:
from keras.preprocessing.image import ImageDataGenerator

#crÃ©ation de nouvelles images deformation rotation d'anciennes images
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

training_set = train_datagen.flow_from_directory('D:/chest-xray-pneumonia/chest_xray/train',
                                                 target_size = (224, 224),
                                                 color_mode='rgb',
                                                 batch_size = 8,
                                                 class_mode = 'binary')

test_set = test_datagen.flow_from_directory('D:/chest-xray-pneumonia/chest_xray/test',
                                            target_size = (224,224),
                                            color_mode='rgb',
                                            batch_size = 1,
                                            class_mode = 'binary')

Using TensorFlow backend.


Found 5216 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


# Modelling

In [2]:
# Importing libraries
from mlxtend.plotting import plot_confusion_matrix
from sklearn.metrics import confusion_matrix
import numpy as np
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import ZeroPadding2D
from keras.layers import BatchNormalization
from keras.layers import Activation
from keras.layers import Dropout

In [3]:
classifier = Sequential()

classifier.add(Conv2D(input_shape=(224,224,3),filters=32,kernel_size=(3,3),padding="same",activation="relu"))
classifier.add(Conv2D(filters=32,kernel_size=(3,3),padding="same", activation="relu"))
classifier.add(BatchNormalization())
classifier.add(MaxPooling2D((2, 2), strides = (2, 2)))


classifier.add(Conv2D(filters=64,kernel_size=(3,3),padding="same",activation="relu"))
classifier.add(Conv2D(filters=64,kernel_size=(3,3),padding="same", activation="relu"))
classifier.add(BatchNormalization())
classifier.add(MaxPooling2D((2, 2), strides = (2, 2)))

classifier.add(Conv2D(filters=128,kernel_size=(3,3),padding="same",activation="relu"))
classifier.add(Conv2D(filters=128,kernel_size=(3,3),padding="same", activation="relu"))
classifier.add(Conv2D(filters=128,kernel_size=(3,3),padding="same", activation="relu"))
classifier.add(BatchNormalization())
classifier.add(MaxPooling2D((2, 2), strides = (2, 2)))

classifier.add(Conv2D(filters=256,kernel_size=(3,3),padding="same",activation="relu"))
classifier.add(Conv2D(filters=256,kernel_size=(3,3),padding="same", activation="relu"))
classifier.add(Conv2D(filters=256,kernel_size=(3,3),padding="same", activation="relu"))
classifier.add(BatchNormalization())
classifier.add(MaxPooling2D((2, 2), strides = (2, 2)))

classifier.add(Conv2D(filters=512,kernel_size=(3,3),padding="same",activation="relu"))
classifier.add(Conv2D(filters=512,kernel_size=(3,3),padding="same", activation="relu"))
classifier.add(Conv2D(filters=512,kernel_size=(3,3),padding="same", activation="relu"))
classifier.add(BatchNormalization())
classifier.add(MaxPooling2D((2, 2), strides = (2, 2)))


classifier.add(Flatten())
classifier.add(Dense(256, activation='relu', name='fc1'))
classifier.add(Dropout(rate=0.5, name='dropout1'))
classifier.add(Dense(128, activation='relu', name='fc2'))
classifier.add(Dropout(rate=0.5, name='dropout2'))
classifier.add(Dense(1, activation='sigmoid', name='fc3'))

In [4]:
classifier.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
from keras.callbacks import ModelCheckpoint, EarlyStopping


checkpoint = ModelCheckpoint("model.h5", monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)

early = EarlyStopping(monitor='val_acc', min_delta=0, patience=20, verbose=1, mode='auto')

history=classifier.fit_generator(steps_per_epoch=100,generator=training_set, validation_data= test_set, validation_steps=10,epochs=100,callbacks=[checkpoint,early])

Epoch 1/100
100/100 [==============================] - 40s 399ms/step - loss: 4.3182 - accuracy: 0.7362 - val_loss: 49.6190 - val_accuracy: 0.6000
Epoch 2/100


c:\users\sylvain\.conda\envs\test\lib\site-packages\keras\callbacks\callbacks.py:707: RuntimeWarning: Can save best model only with val_acc available, skipping.
  'skipping.' % (self.monitor), RuntimeWarning)
c:\users\sylvain\.conda\envs\test\lib\site-packages\keras\callbacks\callbacks.py:846: RuntimeWarning: Early stopping conditioned on metric `val_acc` which is not available. Available metrics are: val_loss,val_accuracy,loss,accuracy
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


100/100 [==============================] - 36s 359ms/step - loss: 2.0478 - accuracy: 0.7850 - val_loss: 54.6277 - val_accuracy: 0.5000
Epoch 3/100
100/100 [==============================] - 35s 351ms/step - loss: 1.1974 - accuracy: 0.8150 - val_loss: 2.0426e-22 - val_accuracy: 0.6000
Epoch 4/100
100/100 [==============================] - 36s 359ms/step - loss: 1.0081 - accuracy: 0.7862 - val_loss: 0.7892 - val_accuracy: 0.7000
Epoch 5/100
100/100 [==============================] - 36s 358ms/step - loss: 0.7729 - accuracy: 0.7987 - val_loss: 9.0152e-29 - val_accuracy: 0.8000
Epoch 6/100
100/100 [==============================] - 35s 355ms/step - loss: 0.4750 - accuracy: 0.8163 - val_loss: 1.9999 - val_accuracy: 0.4000
Epoch 7/100
100/100 [==============================] - 30s 298ms/step - loss: 0.4542 - accuracy: 0.7987 - val_loss: 0.6029 - val_accuracy: 1.0000
Epoch 8/100
100/100 [==============================] - 25s 253ms/step - loss: 0.5355 - accuracy: 0.8200 - val_loss: 1.1667 - va

In [13]:
classifier.save('D:/chest-xray-pneumonia/chest_xray/model3.h5')

In [30]:
from keras.models import load_model
classifier = load_model('D:/chest-xray-pneumonia/chest_xray/model3.h5')

In [14]:
classifier.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 224, 224, 32)      896       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 224, 224, 32)      9248      
_________________________________________________________________
batch_normalization_1 (Batch (None, 224, 224, 32)      128       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 112, 112, 32)      0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 112, 112, 64)      18496     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 112, 112, 64)      36928     
_________________________________________________________________
batch_normalization_2 (Batch (None, 112, 112, 64)     

In [15]:
test_loss, test_score = classifier.evaluate(test_set)

624/624 [==============================] - 16s 26ms/step


In [16]:
test_loss,test_score

(1.4878905858495273e-05, 0.8605769276618958)

In [22]:
preds = classifier.predict_generator(test_set)

In [18]:
orig_test_labes= test_set.classes
orig_test_labes=np.reshape(orig_test_labes,(624,1))

In [26]:
preds[preds<0.5]=0 
preds[preds>=0.5]=1

In [23]:
preds

array([[6.57816231e-01],
       [1.89583465e-01],
       [9.98303413e-01],
       [9.99785244e-01],
       [9.98238325e-01],
       [9.99995232e-01],
       [9.99845266e-01],
       [9.99419212e-01],
       [1.00000000e+00],
       [9.99999762e-01],
       [9.99824464e-01],
       [9.99988794e-01],
       [1.37840286e-02],
       [9.96232688e-01],
       [8.96838546e-01],
       [3.88792902e-01],
       [9.99988198e-01],
       [3.07012582e-03],
       [9.99863625e-01],
       [9.92141128e-01],
       [9.99625087e-01],
       [1.94882862e-02],
       [3.94294523e-02],
       [8.97568703e-01],
       [9.74791050e-01],
       [9.99765337e-01],
       [1.09504022e-01],
       [1.00000000e+00],
       [9.99946713e-01],
       [9.99918938e-01],
       [9.99999762e-01],
       [5.54413378e-01],
       [9.82818186e-01],
       [9.99769270e-01],
       [9.99995351e-01],
       [9.99808729e-01],
       [1.42053068e-02],
       [9.99999404e-01],
       [9.99821961e-01],
       [2.40597084e-01],


In [2]:
import matplotlib.pyplot as plt
cm  = confusion_matrix(orig_test_labes, preds)
plt.figure()
plot_confusion_matrix(cm,figsize=(12,8), hide_ticks=True,cmap=plt.cm.Blues)
plt.xticks(range(2), ['Normal', 'Pneumonia'], fontsize=16)
plt.yticks(range(2), ['Normal', 'Pneumonia'], fontsize=16)
plt.show()

NameError: name 'confusion_matrix' is not defined

In [21]:
tn, fp, fn, tp = cm.ravel()

precision = tp/(tp+fp)
recall = tp/(tp+fn)

print("Recall of the model is {:.2f}".format(recall))
print("Precision of the model is {:.2f}".format(precision))

Recall of the model is 0.73
Precision of the model is 0.62


In [31]:
import matplotlib.pyplot as plt
plt.plot(classifier.history["acc"])
plt.plot(classifier.history['val_acc'])
plt.plot(classifier.history['loss'])
plt.plot(classifier.history['val_loss'])
plt.title("model accuracy")
plt.ylabel("Accuracy")
plt.xlabel("Epoch")
plt.legend(["Accuracy","Validation Accuracy","loss","Validation Loss"])
plt.show()

AttributeError: 'Sequential' object has no attribute 'history'